# Fitting

In [1]:
import sys
sys.path.insert(0, '../../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle as pkl
import tensorflow as tf

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from kraus_channels import KrausMap
from loss_functions import ProbabilityMSE, ProbabilityRValue, channel_fidelity_loss
from optimization import ModelSPAM, ModelQuantumMap, Logger, model_saver
from quantum_channel import DilutedChannel, ChoiMapStatic, channel_fidelity
from quantum_tools import apply_unitary, measurement, resample, state_fidelity, reshuffle
from experimental import counts_to_probs, generate_pauliInput_circuits, generate_pauli_circuits, marginalize_counts
from spam import SPAM, InitialState, POVMwQR as POVM, IdealPOVM, IdealInit, povm_fidelity, CorruptionMatrix, IdealSPAM
from synthetic_data import generate_spam_data, generate_map_data
from utils import saver, loader, train_val_split
from quantum_circuits import pqc_basic
from spectrum import channel_spectrum, complex_spacing_ratio, Spectrum, coat_spectrum, spectrum_to_angular, spectrum_to_radial
from qiskit.circuit.library import iSwapGate


#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
def model_to_csr(model_list):
    spectrum_list = []
    csr_integrable_list = []
    for model in model_list:
        channel = model.channel
        spectrum = channel_spectrum(channel)

        csr = complex_spacing_ratio(spectrum)

        spectrum_list.append(spectrum)
        csr_integrable_list.append(csr)

    return csr_integrable_list

## Retrieve CSR

### L = 5

In [3]:
model_list = pkl.load(open(f'models/integrable_model_4_L=5.model', 'rb'))

superoperator_list = [model.channel.superoperator for model in model_list]

for i, superoperator in enumerate(superoperator_list):
    np.savetxt(f'data/superoperator_L=5/superoperator{i}', superoperator, fmt='%f')

In [15]:
model_list = pkl.load(open(f'models/integrable_model_4_L=10.model', 'rb'))

superoperator_list = [model.channel.superoperator for model in model_list]

for i, superoperator in enumerate(superoperator_list):
    np.savetxt(f'data/superoperator_L=10/superoperator{i}', superoperator, fmt='%f')

In [4]:
model_list = pkl.load(open(f'models/integrable_model_4_L=5_rerun.model', 'rb'))

superoperator_list = [model.channel.superoperator for model in model_list]

for i, superoperator in enumerate(superoperator_list):
    np.savetxt(f'data/superoperator_rerun', superoperator, fmt='%f')

In [4]:
model1 = pkl.load(open(f'models/integrable_model_4_L=5.model', 'rb'))[1]
model2 = pkl.load(open(f'models/integrable_model_4_L=5_rerun.model', 'rb'))[0]


In [5]:
channel_fidelity(model1.channel, model2.channel)

<tf.Tensor: shape=(), dtype=float64, numpy=0.9651508388726432>

In [6]:
map_ideal_list = pkl.load(open("ideal_maps.pkl", "rb"))

In [16]:
np.sqrt(channel_fidelity(model1.channel, map_ideal_list[1]))

0.5190658817718946

In [17]:
np.sqrt(channel_fidelity(model2.channel, map_ideal_list[1]))

0.5110929294578674